# Open Science for notebooks

This use case will address Open Science for data and code sharing using platforms such as Zenodo and OpenAIRE and making the entire research process more transparent and accessible, promoting collaboration, and maximizing the impact of research. It will try to answer the follwing research questions: (1) How can Open Science and Digital Commons be addressed by means of notebooks in the ECCCH? (2) How can we showcase, extend and improve the ECCCH to fully support Open Science and Digital Commons in order to engage with the public?

This approach employs a workflow as shown in the following picture.

<img src="imgs/workflow.png" width="100%">

### OpenAIRE Graph

The [OpenAIRE Graph](https://graph.openaire.eu/what-is-the-openaire-graph) is a free and open resource that brings together and interlinks hundreds of millions of metadata records from over 100k data sources trusted by researchers. The project broke ground in 2012 being one of the first research knowledge graphs and has now grown into one of the world's largest and is the authoritative source for the [European Open Science Cloud (EOSC)](https://eosc.eu/). It enables researchers, communities, institutions, companies, and citizens sharing research products and related information.

In this example, we will try to show how to reuse OpenAIRE as a repository and protoype an integration with the ECCCH.

OpenAire Graph can be accessed using its API. See the following example to retrieve research products which correspond to data, publications and software according to the [OpenAIRE data model](https://graph.openaire.eu/docs/data-model/).

In [2]:
import requests

url = "https://api.openaire.eu/graph/v2/researchProducts"
params = {
    "search": "OpenAIRE Graph",
    "type": "publication",
    "page": 1,
    "pageSize": 10,
    "sortBy": "relevance DESC"
}
headers = {
    "accept": "application/json"
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    #print(data)
else:
    print(f"Failed to retrieve data: {response.status_code}")

### Filtering search results

Filters can be used to narrow down the search results based on specific criteria. Filters are provided as query parameters in the request URL. See, for example:

- Get all research products that contain the word "covid", sorted by popularity in descending order: [https://api.openaire.eu/graph/v2/researchProducts?search=covid&sortBy=popularity%20DESC](https://api.openaire.eu/graph/v2/researchProducts?search=covid&sortBy=popularity%20DESC)
- Get all publications that are published after 2020-01-01: https://api.openaire.eu/graph/v2/researchProducts?type=publication&fromPublicationDate=2020-01-01
- Get the organization with the ROR id https://ror.org/05t8bcz72 (University of Alicante): https://api.openaire.eu/graph/v1/organizations?pid=https://ror.org/05t8bcz72

We can see the list of parameters that we can use in this [link](https://graph.openaire.eu/docs/apis/graph-api/searching-entities/filtering-search-results#available-parameters). 

Some examples of useful parameters are the page (Integer) and pageSize (Integer in the range [1, 100]) to paginate the results. See, for example, how to retrieve the top 100 most influential research products that contain the phrase "knowledge graphs":

- [https://api.openaire.eu/graph/v2/researchProducts?search=%22knowledge%20graphs%22&page=1&pageSize=100&sortBy=influence%20DESC](https://api.openaire.eu/graph/v2/researchProducts?search=%22knowledge%20graphs%22&page=1&pageSize=100&sortBy=influence%20DESC)

#### Extraction

Now we transform the human-readable text to a CSV so the data can be analysed and adapted easily. For this, we create a CSV file, each row containing a record.

In [3]:
import requests
import json

url = "https://api.openaire.eu/graph/v2/researchProducts"
params = {
    "search": "knowledge graph",
    "type": "publication",
    "page": 1,
    "pageSize": 100,
    "sortBy": "influence DESC"
}
headers = {
    "accept": "application/json"
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    #print(data)
    with open('data/openaire/metadata-openaire.json', 'w') as jsonfile:
        json.dump(data, jsonfile)
else:
    print(f"Failed to retrieve data: {response.status_code}")

The result is a JSON file:

```
{'header': {'numFound': 55360, 'queryTime': 74, 'page': 1, 'pageSize': 100}, 'results': [{'authors': [{'fullName': 'M, Kanehisa', 'name': 'M', 'surname': 'Kanehisa', 'rank': 1, 'pid': None}, {'fullName': 'S, Goto', 'name': 'S', 'surname': 'Goto', 'rank': 2, 'pid': None}], 'openAccessColor': 'gold', 'publiclyFunded': False, 'type': 'publication', 'language': {'code': 'und', 'label': 'Undetermined'}, 'countries': None, 'subjects': [{'subject': {'scheme': 'FOS', 'value': '0301 basic medicine'}, 'provenance': None}, {'subject': {'scheme': 'FOS', 'value': '0303 health sciences'}, 'provenance': None}, {'subject': {'scheme': 'FOS', 'value': '03 medical and health sciences'}, 'provenance': None}, {'subject': {'scheme': 'keyword', 'value': 'Genome'}, 'provenance': None}, {'subject': {'scheme': 'keyword', 'value': 'Databases, Factual'}, 'provenance': None}, {'subject': {'scheme': 'keyword', 'value': 'Japan'}, 'provenance': None}, {'subject': {'scheme': 'keyword', 'value': 'Animals'}, 'provenance': None}, {'subject': {'scheme': 'keyword', 'value': 'Gene Expression'}, 'provenance': None}, {'subject': {'scheme': 'keyword', 'value': 'Humans'}, 'provenance': None}, {'subject': {'scheme': 'keyword', 'value': 'Information Storage and Retrieval'}, 'provenance': None}, {'subject': {'scheme': 'keyword', 'value': 'Proteins'}, 'provenance': None}], 'mainTitle': 'KEGG: Kyoto Encyclopedia of Genes and Genomes', 'subTitle': None, 'descriptions': ['KEGG (Kyoto Encyclopedia of Genes and Genomes) is a knowledge base for systematic analysis of gene functions, linking genomic information with higher order functional information. The genomic information is stored in the GENES database, which is a collection of gene catalogs for all the completely sequenced genomes and some partial genomes with up...```

#### Configuration of the runtime environment 

Let's explore how to use Retrieval-Augmented Generation (RAG) using the descriptions of the research items. We will be able to ask questions about knowledge graphs using a prompt.

### Load embedding model and database

We define a folder to store the vector dabatase which is based on Chroma, an open-source search and retrieval database for AI applications. For testing purposes, we employ a pre-trained embedding model.

In [5]:
# Import embedding and vector database libraries
from sentence_transformers import SentenceTransformer
import chromadb
import os
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from chromadb.utils import embedding_functions

res_folder = "data/openaire/"

# Configuration for vector database
db_collection_name = "zenodo-eccch"  # Name for our vector collection
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"  # Pre-trained embedding model
path_chroma = os.path.join(res_folder, "chroma_db")  # Path to store the vector database

print(path_chroma)

data/openaire/chroma_db


### Create word embedding using the descriptions as input

In [6]:
import json

# Initialize ChromaDB client and collection
client = chromadb.PersistentClient(path=path_chroma)

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
     model_name=embedding_model
)

collection = client.get_or_create_collection(name=db_collection_name,
                                             embedding_function=embedding_func)

# Load the sentence transformer model for creating embeddings
model = SentenceTransformer(embedding_model, token=False)

# Get all cleaned text descriptions
with open('data/openaire/metadata-openaire.json', encoding="utf-8") as jsonfile:
    data = json.load(jsonfile)
    for item in data["results"]:
        #print (item)
        lines = [line.strip() for line in item["descriptions"]]
        #print(lines)

        embeddings = model.encode(lines, show_progress_bar=True, convert_to_numpy=True)
    
        # Add embeddings and metadata to ChromaDB
        #try:
        collection.add(
            ids=[f"{[item['id']]}_{i}" for i in range(len(lines))],
            embeddings=embeddings.tolist(),
            documents=lines,
            #metadatas=[{"filename": fname, "date": datestr, "line": i, "url": url} for i, url in enumerate(urls)]
            metadatas=[{"id": item["id"], "title": item["mainTitle"]} for i in range(len(lines))]
        )
        #except ValueError:
        #    print(f"⚠️ Skipped {item['id']} due to ValueError")

print("✅ Indexed all lines into Chroma!")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Indexed all lines into Chroma!


### Now we define a new function enabling the semantic search using vector embeddings

In [8]:
def semantic_search(query, n_results=2):
    """Perform semantic search on the corpus using vector embeddings.
    
    Args:
        query: The search query as a string
        n_results: Number of results to return (default: 5)
        
    Returns:
        Dictionary containing search results from ChromaDB
    """
    # Load the embedding model
    model = SentenceTransformer(embedding_model)

    # Connect to the vector database
    client = chromadb.PersistentClient(path=path_chroma)
    collection = client.get_or_create_collection(name=db_collection_name)

    # Convert query to embedding vector
    query_emb = model.encode([query], convert_to_numpy=True).tolist()
    
    # Search for similar content in the database
    results = collection.query(
        query_embeddings=query_emb,
        n_results=n_results
    )
    print(results)
    # Display results with metadata
    for text, meta in zip(results["documents"][0], results["metadatas"][0]):
        print(f"{meta['id']}")
        print(f"→ {text[:]}\n")

    return results

### We finally test the semantic search function

In [9]:
# Example semantic search query
# This demonstrates finding content related to knowlege graphs without requiring exact keyword matches
res = semantic_search("What is a knowledge graph?", 2)

{'ids': [["['doi_dedup___::2a50b43777588079b733c9d5d2a257c1']_0", "['doi_dedup___::2107e4a1710136edf62683a9cc063f0b']_0"]], 'embeddings': None, 'documents': [['<jats:p>In this article, we provide a comprehensive introduction to knowledge graphs, which have recently garnered significant attention from both industry and academia in scenarios that require exploiting diverse, dynamic, large-scale collections of data. After some opening remarks, we motivate and contrast various graph-based data models, as well as languages used to query and validate knowledge graphs. We explain how knowledge can be represented and extracted using a combination of deductive and inductive techniques. We conclude with high-level future research directions for knowledge graphs.</jats:p>', 'In the recent years, different Web knowledge graphs, both free and commercial, have been created. While Google coined the term “Knowledge Graph” in 2012, there are also a few openly available knowledge graphs, with DBpedia, Y

As shown in this example, the OCR output provided by the model could be used to apply additional techniques such as Named-Entity Recoginition and Entity Linking to enrich the original metadata provided by the institution. We present this work as a preliminary approach and additional is required in this sense.

### Integration with the ECCCH

[ECHOES](https://www.echoes-eccch.eu/faq/) is building a federated Knowledge Graph to allow for high level integration of resources. It will also serve as an entry point for all queries and requests related to any kind of information available within the Cultural heritage Cloud. The Knowledge Graph will use the proposed Heritage Digital Twin Ontology (HDTO) to unify descriptions and facilitate query and navigation. The current version of the ECHOES HDTO is available [here](https://www.echoes-eccch.eu/wp-content/uploads/2025/06/ECHOES_HDT_Ontology.pdf). The main vocabulary employed to describe the resources is [CIDOC-CRM](https://cidoc-crm.org/).

The following illustration shows how we modelled the outputs of this work in order to be integrated into the ECCCH.
<img width="80%" src="imgs/eccch-integration-steps.png">

And the following picture shows how we modelled the data using the vocabularies and ontologies. The class prov:Entity represents the code in the form of Jupyter Notebook which was generated by means of a prov:Activity, describing the work in this code, using a distribution of a dataset as an input (txt), and generating another distribution (ttl). The distributions are part of a dataset, which is also part of a catalog that was published by an organization.

<img width="80%" src="imgs/data-model-cidoc.png">

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import FOAF, RDF, RDFS, DCTERMS, VOID, DC, SKOS, OWL, XSD
import datetime

g = Graph()
g.bind("foaf", FOAF)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
g.bind("void", VOID)
g.bind("skos", SKOS)
g.bind("owl", OWL)

schema = Namespace("https://schema.org/")
g.bind("schema", schema)

dcat = Namespace("http://www.w3.org/ns/dcat#")
g.bind("dcat", dcat)

wd = Namespace("http://www.wikidata.org/entity/")
g.bind("wd", wd)

cidoc_crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
g.bind("cidoc-crm", cidoc_crm)

prov = Namespace("http://www.w3.org/ns/prov#")
g.bind("prov", prov)

domain = 'https://example.org/'
domainLanguage = domain + 'map/'

##### We add the metadata of the dataset

In [36]:
# First, we create all the required URIS
openaire_catalog = URIRef(domain + "catalog/openaire")
openaire_org = URIRef(domain + "organization/openaire")
openaire_dataset = URIRef(domain + "dataset/openaire")
openaire_json = URIRef(domain + "distribution/openaire-json")
openaire_ttl = URIRef(domain + "distribution/openaire-ttl")

In [37]:
# We describe the dataset
g.add((openaire_catalog, RDF.type, schema.Dataset))
g.add((openaire_catalog, RDF.type, dcat.catalog))
g.add((openaire_catalog, RDFS.label, Literal("OpenAIRE Knowledge Graph")))
g.add((openaire_catalog, schema.url, URIRef("https://graph.openaire.eu/")))
g.add((openaire_catalog, FOAF.homepage, URIRef("https://graph.openaire.eu/")))
g.add((openaire_catalog, schema.description, Literal("The OpenAIRE Graph is a free and open resource that brings together and interlinks hundreds of millions of metadata records from over 100k data sources trusted by researchers.")))
g.add((openaire_catalog, schema.name, Literal("The OpenAIRE Graph is a free and open resource that brings together and interlinks hundreds of millions of metadata records from over 100k data sources trusted by researchers.")))
g.add((openaire_catalog, DCTERMS.title, Literal("The OpenAIRE Graph is a free and open resource that brings together and interlinks hundreds of millions of metadata records from over 100k data sources trusted by researchers.")))
g.add((openaire_catalog, DCTERMS.publisher, URIRef(openaire_org))) # relation dataset-publisher
g.add((openaire_catalog, DC.title, Literal("OpenAIRE")))
g.add((openaire_catalog, schema.license, URIRef('https://creativecommons.org/licenses/by/4.0/')))
g.add((openaire_catalog, dcat.dataset, openaire_dataset))

now = datetime.datetime.now()
g.add((openaire_catalog, schema.dateCreated, Literal(str(now)[:10])))

<Graph identifier=Ncaa87101cd604e7f93c7353f0c429988 (<class 'rdflib.graph.Graph'>)>

In [38]:
# We describe OpenAIRE
g.add((openaire_org, RDF.type, FOAF.Organization))
g.add((openaire_org, RDFS.label, Literal("OpenAIRE")))
g.add((openaire_org, FOAF.homepage, URIRef("https://graph.openaire.eu/")))

<Graph identifier=Ncaa87101cd604e7f93c7353f0c429988 (<class 'rdflib.graph.Graph'>)>

In [39]:
# We describe the dataset
g.add((openaire_dataset, RDF.type, dcat.Dataset))
g.add((openaire_dataset, DCTERMS.title, Literal("Top 100 most influential research products in OpenAIRE that contain the phrase knowledge graphs", lang="en")))
g.add((openaire_dataset, dcat.keyword, Literal("Research products")))
g.add((openaire_dataset, dcat.keyword, Literal("Knowledge graphs")))
g.add((openaire_dataset, dcat.keyword, Literal("Data")))
g.add((openaire_dataset, DCTERMS.issued, Literal(str(now)[:10])))
g.add((openaire_dataset, DCTERMS.language, URIRef("http://id.loc.gov/vocabulary/iso639-1/en")))
g.add((openaire_dataset, dcat.distribution, URIRef(openaire_json)))
g.add((openaire_dataset, dcat.distribution, URIRef(openaire_ttl)))

<Graph identifier=Ncaa87101cd604e7f93c7353f0c429988 (<class 'rdflib.graph.Graph'>)>

In [40]:
# We describe the distributions JSON and TTL 
g.add((openaire_json, RDF.type, dcat.Distribution))
g.add((openaire_json, dcat.downloadURL , URIRef("https://raw.githubusercontent.com/hibernator11/eccch-use-cases/refs/heads/main/data/openaire/metadata-openaire.json")))
g.add((openaire_json, DCTERMS.title, Literal("JSON distribution of OpenAIRE results", lang="en")))
g.add((openaire_json, DCTERMS.title, Literal("Distribución en JSON del conjunto de datos de la búsqueda de OpenAIRE", lang="es")))
g.add((openaire_json, dcat.mediaType, URIRef("http://www.iana.org/assignments/media-types/application/json")))
g.add((openaire_json, dcat.byteSize, Literal('1000000', datatype=XSD.integer)))

g.add((openaire_ttl, RDF.type, dcat.Distribution))
g.add((openaire_ttl, dcat.downloadURL , URIRef("https://raw.githubusercontent.com/hibernator11/eccch-use-cases/refs/heads/main/data/openaire/dataset_openaire.ttl")))
g.add((openaire_ttl, DCTERMS.title, Literal("TTL distribution of of OpenAIRE results", lang="en")))
g.add((openaire_ttl, DCTERMS.title, Literal("Distribución en TTL del conjunto de datos de la búsqueda de OpenAIRE", lang="es")))
g.add((openaire_ttl, dcat.mediaType, URIRef("http://www.iana.org/assignments/media-types/application/n-triples")))
g.add((openaire_ttl, dcat.byteSize, Literal('360000', datatype=XSD.integer)))

<Graph identifier=Ncaa87101cd604e7f93c7353f0c429988 (<class 'rdflib.graph.Graph'>)>

### Now we link the notebooks and the distributions of the dataset created

In [41]:
openscience_work = URIRef(domain + "openscience-work/openaire")
notebooks = URIRef(domain + "notebooks/openaire")
g.add((notebooks, RDF.type, prov.Entity))
g.add((notebooks, DCTERMS.title, Literal("Open Science for notebooks", lang="en")))
g.add((notebooks, prov.wasGeneratedBy, URIRef(openscience_work)))
g.add((notebooks, dcat.mediatype, URIRef("https://www.iana.org/assignments/media-types/application/x-ipynb+json")))

now = datetime.datetime.now()
g.add((openscience_work, RDF.type, prov.Activity))
g.add((openscience_work, prov.startedAtTime, Literal(str(now)[:10])))
g.add((openscience_work, prov.used, URIRef(openaire_json)))
g.add((openscience_work, prov.generated, URIRef(openaire_ttl)))
g.add((openscience_work, prov.endedAtTime, Literal(str(now)[:10])))

<Graph identifier=Ncaa87101cd604e7f93c7353f0c429988 (<class 'rdflib.graph.Graph'>)>

#### Read the JSON file 

In [42]:
import json

with open('data/openaire/metadata-openaire.json', newline='') as jsonfile:
    data = json.load(jsonfile)
    for item in data["results"]:
        print (item["id"])    

doi_dedup___::f1e1c7b110713cd86039eb5281c58f43
doi_dedup___::7025799718bf197e28e7875600f282e9
doi_dedup___::7e186b2eb7361c33e5e4cba8460b8f01
doi_dedup___::eaad7f81ecd18b541859cb0db923e42f
doi_dedup___::2c7da58890b20821e6351000edd30805
doi_dedup___::aeb32f00d0a525cd4488f53dc4107f8f
doi_dedup___::878648658ca310f0a6858d2a6a7d51e8
doi_dedup___::c03569e68485ef036ccde4d18d4f7670
doi_dedup___::79e99f1580af46a0d1924fc20210c00a
doi_dedup___::fe91e342301d8bd98c80f2e3b5c7869d
doi_dedup___::42448ae06e26c65bcdc583b22c3547b3
doi_dedup___::f89a36bbde12d423abc1738a7d6d1bf2
doi_dedup___::71104cfc619c0368305ce9e70bec3502
doi_dedup___::2d8f6135f6f011739308e5b3a5246517
doi_dedup___::f5a2955abae03c612afb155f176ee0b9
doi_dedup___::d23f9c1c67f85b164480c60179d2f9c4
doi_dedup___::5c1bf0097cb2ac2a1ec4818403289767
doi_________::d3deb4305ebba8220a3ce4e918a08d40
doi_dedup___::52e8fad047b1e6e7bb383bbedc327c5e
doi_dedup___::e61d23328646279a6076782ad70e3680
doi_dedup___::21b0d3cf169ebe432ad1b4f4f4d9fcac
doi_dedup___:

#### Store the data

In [43]:
g.serialize(destination="data/openaire/dataset_openaire.ttl")

<Graph identifier=Ncaa87101cd604e7f93c7353f0c429988 (<class 'rdflib.graph.Graph'>)>

#### Now we can query the graph using SPARQL

In [44]:
print('##### Properties:')

# Query the data in g using SPARQL
q = """
    SELECT distinct ?prop
    WHERE {
        ?s ?prop ?o .
    }
"""

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r["prop"])

##### Properties:
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2000/01/rdf-schema#label
http://www.w3.org/ns/dcat#byteSize
http://purl.org/dc/terms/title
http://xmlns.com/foaf/0.1/homepage
https://schema.org/description
http://www.w3.org/ns/dcat#dataset
http://www.w3.org/ns/dcat#distribution
http://www.w3.org/ns/prov#endedAtTime
http://www.w3.org/ns/prov#wasGeneratedBy
https://schema.org/name
http://purl.org/dc/terms/language
https://schema.org/url
https://schema.org/dateCreated
http://www.w3.org/ns/dcat#keyword
http://www.w3.org/ns/dcat#downloadURL
http://www.w3.org/ns/prov#used
http://purl.org/dc/terms/issued
https://schema.org/license
http://www.w3.org/ns/dcat#mediaType
http://www.w3.org/ns/prov#generated
http://purl.org/dc/terms/publisher
http://purl.org/dc/elements/1.1/title
http://www.w3.org/ns/prov#startedAtTime
http://www.w3.org/ns/dcat#mediatype


As an example we can retrieve the metadata of our dataset

In [45]:
print('##### Dataset information:')

# Query the data in g using SPARQL
q = """
    PREFIX schema: <https://schema.org/>
    SELECT distinct ?p ?o
    WHERE {
        ?s a schema:Dataset .
        ?s ?p ?o
    }
"""

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r["p"] + ": " + r["o"])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type:  does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: https://schema.org/Dataset does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: https://schema.org/Dataset does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type:  does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: http://www.w3.org/ns/dcat#catalog does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: http://www.w3.org/ns/dcat#catalog does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/2000/01/rdf-schema#label:  does not look like a valid URI, trying to serialize this will break.
http://www.w3.org/2000/01/rdf-schema#label: OpenAIRE K

##### Dataset information:
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: https://schema.org/Dataset
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: http://www.w3.org/ns/dcat#catalog
http://www.w3.org/2000/01/rdf-schema#label: OpenAIRE Knowledge Graph
https://schema.org/url: https://graph.openaire.eu/
http://xmlns.com/foaf/0.1/homepage: https://graph.openaire.eu/
https://schema.org/description: The OpenAIRE Graph is a free and open resource that brings together and interlinks hundreds of millions of metadata records from over 100k data sources trusted by researchers.
https://schema.org/name: The OpenAIRE Graph is a free and open resource that brings together and interlinks hundreds of millions of metadata records from over 100k data sources trusted by researchers.
http://purl.org/dc/terms/title: The OpenAIRE Graph is a free and open resource that brings together and interlinks hundreds of millions of metadata records from over 100k data sources trusted by researchers.
http://purl.or

#### Finally, we can use the ECCCH API to publish the data generated.

In [1]:
## To Be done!
# Integration with the ECCCH once the API and fina data model is available

### Publication & dissemination

This step involves the publication of the results obtained including the dataset and this notebook in different platforms such as the Social Sciences and Humanities Open Marketplace and Zenodo.


As an example, we will use the sandbox service of Zenodo. Note that if you want to use this code for production purposes, it is required to update the URL. First, we need to create an access token in this [link](https://zenodo.org/account/settings/applications/tokens/new/). Note that we also need a token for the sandbox Zenodo.

In [43]:
# https://developers.zenodo.org/
import requests
ACCESS_TOKEN = 'ChangeMe'

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}
r = requests.post('https://sandbox.zenodo.org/api/deposit/depositions',
                   json={},
                   headers=headers)
r.status_code
r.json()

{'created': '2026-01-12T17:41:07.632652+00:00',
 'modified': '2026-01-12T17:41:07.740296+00:00',
 'id': 424692,
 'conceptrecid': '424691',
 'metadata': {'access_right': 'open',
  'prereserve_doi': {'doi': '10.5281/zenodo.424692', 'recid': 424692}},
 'title': '',
 'links': {'self': 'https://sandbox.zenodo.org/api/deposit/depositions/424692',
  'html': 'https://sandbox.zenodo.org/deposit/424692',
  'badge': 'https://sandbox.zenodo.org/badge/doi/.svg',
  'files': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/files',
  'bucket': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c',
  'latest_draft': 'https://sandbox.zenodo.org/api/deposit/depositions/424692',
  'latest_draft_html': 'https://sandbox.zenodo.org/deposit/424692',
  'publish': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/actions/publish',
  'edit': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/actions/edit',
  'discard': 'https://sandbox.zenodo.org/api/deposit/depos

Now, let’s upload a new file:

In [44]:
bucket_url = r.json()["links"]["bucket"]
deposition_id = r.json()["id"]

First, we create a zip file with the notebook and the requirements.txt file:

In [45]:
from zipfile import ZipFile

# List of files to include in the archive
file_list = ["Open-Science.ipynb", "requirements.txt"]

# Create ZIP file and write files into it
with ZipFile("output.zip", "w") as zipf:
   for file in file_list:
      zipf.write(file)

Then, we call the API:

In [47]:
filename = "output.zip"
path = "%s" % filename
headers = {'Authorization': f'Bearer {ACCESS_TOKEN}'}

''' 
The target URL is a combination of the bucket link with the desired filename
seperated by a slash.
'''
with open(path, "rb") as fp:
    r = requests.put(
        "%s/%s" % (bucket_url, filename),
        data=fp,
        headers=headers,
    )
r.json()

{'created': '2026-01-12T17:41:16.912746+00:00',
 'updated': '2026-01-12T17:41:17.052565+00:00',
 'version_id': 'f8cd6099-25aa-4105-aa41-91921a295466',
 'key': 'output.zip',
 'size': 28194,
 'mimetype': 'application/zip',
 'checksum': 'md5:4df29214004c115c3e7a0ab4a433da97',
 'is_head': True,
 'delete_marker': False,
 'links': {'self': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip',
  'version': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip?version_id=f8cd6099-25aa-4105-aa41-91921a295466',
  'uploads': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip?uploads=1'}}

We can also add metadata to the record:

In [49]:
data = {
     'metadata': {
         'title': 'Open Science for notebooks',
         'upload_type': 'software',
         'description': 'This use case will address Open Science for data and code sharing using platforms such as Zenodo and OpenAIRE and making the entire research process more transparent and accessible, promoting collaboration, and maximizing the impact of research',
         'creators': [{'name': 'Candela, Gustavo',
                       'affiliation': 'University of Alicante'}]
     }
 }
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}
r = requests.put('https://sandbox.zenodo.org/api/deposit/depositions/%s' % deposition_id,
                  data=json.dumps(data),
                  headers=headers)
r.status_code

200

The last step is the publication:

In [50]:
headers = {'Authorization': f'Bearer {ACCESS_TOKEN}'}
r = requests.post('https://sandbox.zenodo.org/api/deposit/depositions/%s/actions/publish' % deposition_id,
                      headers=headers)
r.status_code
# 202

202

And now we can see the result in Zenodo:

<img src="imgs/zenodo-publication.png" width="70%">

We can reproduce the same with additional platforms such as [Wikidata](https://www.wikidata.org/) and the [Social Sciences and Humanities Open Marketplace](https://marketplace.sshopencloud.eu/about/api-documentation)

In the particular case of Wikidata, existing [python libraries](https://www.mediawiki.org/wiki/Manual:Pywikibot/Wikidata) can be used to extract and create entities.

### References

- Candela, G., Rosiński, C., & Margraf, A. (2025). A reproducible framework to publish and reuse Collections as data: the case of the European Literary Bibliography (Version 4, Vol. 965, Issue 170). Transformations: A DARIAH Journal . https://doi.org/10.46298/transformations.14729
- Gustavo Candela, Javier Pereda, Dolores Sáez, Pilar Escobar, Alexander Sánchez, Andrés Villa Torres, Albert A. Palacios, Kelly McDonough, and Patricia Murrieta-Flores. 2023. An Ontological Approach for Unlocking the Colonial Archive. J. Comput. Cult. Herit. 16, 4, Article 74 (December 2023), 18 pages. https://doi.org/10.1145/3594727
- https://developers.zenodo.org/#quickstart-upload
- https://www.echoes-eccch.eu/wp-content/uploads/2025/06/ECHOES_HDT_Ontology.pdf
- https://marketplace.sshopencloud.eu/about/api-documentation
- https://www.wikidata.org/
- https://cidoc-crm.org/sites/default/files/CRMdigv4.0.pdf
- https://www.w3.org/TR/prov-o/